In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import imageio
from io import BytesIO
import imgaug as ia
import imgaug.augmenters as iaa


ia.seed(1)


In [16]:
imgaug_seq = iaa.SomeOf((1, 3), [
    # arithmetic
    iaa.SomeOf((1, 3), [
        iaa.JpegCompression(compression=(30, 50)),
        # iaa.imgcorruptlike.Saturate(severity=(1,3)),
        iaa.MultiplyElementwise((0.5, 1.5)),
        iaa.MultiplyElementwise((0.5, 1.5), per_channel=0.5),
        iaa.Multiply((0.5, 1.5)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)),
    ]),
    
    # blur, resize, quantize, convo (sharpen)
    iaa.OneOf([
        iaa.OneOf([
            iaa.KMeansColorQuantization(),
            iaa.UniformColorQuantization(),
        ]),
        iaa.GaussianBlur(sigma=(0.0, 1.0)),
        # iaa.AveragePooling(2),
        iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.8, 1.3)),
    ]),
    
    # color and contrast
    iaa.SomeOf((1, 2), [
        iaa.OneOf([
            iaa.AddToBrightness((-20, 20)),
            iaa.MultiplyBrightness((0.75, 1.25)),
        ]),
        iaa.OneOf([
            iaa.GammaContrast((0.75, 1.5)),
            iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6)),
            iaa.LogContrast(gain=(0.7, 1.3)),
            iaa.LinearContrast((0.6, 1.3)),
        ]),
    ]),
])

imgaug_affine = iaa.SomeOf((3, 4), [
    # geometric (flip, translate/stretch, shear, rotate)
    iaa.Fliplr(),
    iaa.Affine(translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}),
    iaa.Affine(shear=(-16, 16)),
    iaa.Affine(rotate=(-20, 20)),
])

In [17]:
imgaug_cartoon = iaa.Cartoon(blur_ksize=(2,3), segmentation_size=(0.85, 1.05),
                             saturation=(1.4, 2.0), edge_prevalence=1.0)

# iaa.CLAHE(tile_grid_size_px=(3, 21))

augmentor = {
    'a': imgaug_seq, 
    'b': imgaug_cartoon
}




In [18]:

# You are iterating over enumerate(img_aug), which is not a list of augmented images, 
# but only the one last image that you loaded in your loop (i.e. you are saving each 
# row of the last image as its own image). 
# Something like that should work:

dataset = 'a1'
out_dataset = 'b'
in_path = os.path.join('dataset-' + dataset)
out_path = os.path.join('dataset-' + dataset + out_dataset)
# in_path = os.path.join('validation')
# out_path = os.path.join('val-cartoon')

ia.seed(73451)

# plt.figure(figsize=(10, 10))
# for images, _ in train_ds.take(1):
#     for i in range(7):
#         augmented_images = imgaug_seq(images)
#         ax = plt.subplot(1, 7, i + 1)
#         plt.imshow(augmented_images[0].numpy().astype("uint8"))
#         plt.axis("off")

# seq = iaa.Sequential([
#     iaa.Fliplr(0.5),
#     iaa.Crop(percent=(0, 0.1)),
#     iaa.Affine(rotate=(-25,25))
# ], random_order=True)


for folder in ['fire', 'nofire']:
    if not os.path.exists(os.path.join(out_path, folder)):
        os.mkdir(os.path.join(out_path, folder))
    for fname in os.listdir(os.path.join(in_path, folder)):
        img = imageio.imread(os.path.join(in_path, folder, fname))
        filename = fname[:fname.rfind('.')]
        extension = fname[fname.rfind('.'):]
        for i in range(3 + (-2 if out_dataset == 'b' else 0)):
            img_aug = augmentor[out_dataset].augment_image(img)
            img_aug = imgaug_affine.augment_image(img_aug)
            out_filename = filename + "_ia%02d" % (i,) + extension
            PIL.Image.fromarray(img_aug).save(os.path.join(out_path, folder, out_filename))
        # break

# PIL.Image.fromarray(img_aug).show()


/tmp/ipykernel_6382/3185850149.py:34: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(os.path.join(in_path, folder, fname))


In [19]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds2.take(1):
#     for i in range(7):
#         ax = plt.subplot(1, 7, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")
